**This notebook contains all the code for running Sparse Identification of Non Linear Dynamics (SINDy) using PySINDy. There are two sections: Models using Steady State Dynamics Data and Models using Transient Dynamics Data. Further, the model built using Transient Data is used to simulate further to predict steady state and assess the model's abilities to learn the true dynamics and steady state stability.**

**Note: The data for transients and steady state are different and thus have to be imported with their respective paths in the drive.*

## **IMPORTS**

In [29]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from scipy.integrate import odeint
from matplotlib import animation
import matplotlib
import time
import sys
import math
from matplotlib import animation, rc, rcParams
from IPython.display import HTML
import time
import csv
from IPython.display import display
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
import torch.nn.functional as F
import random
import numpy as np
import matplotlib
import pandas as pd
import dill
from PIL import Image
import random
import torchvision
import torch.optim as optim
import matplotlib.pyplot as plt
import plotly.express as plotx
!pip install pysindy
import pysindy as ps

## **SINDy MODEL FOR STEADY STATE**

### **SETTING UP DATA**

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:

file_name = '/content/drive/My Drive/522_HW4_Files/swarm_03242020_3000steps.csv'
my_data = []
with open(file_name) as csvDataFile:
  csvReader = csv.reader(csvDataFile)
  for row in csvReader:
    my_data.append(row)
pos_from_file = []
vel_from_file =[]
pos_line = []
vel_line = []

for i in range(len(my_data)):
  for j in range(int(len(my_data[0])/2)):
    try:
      pos_line_str = my_data[i][j*2].lstrip('[').rstrip(']').split()
      vel_line_str = my_data[i][j*2+1].lstrip('[').rstrip(']').split()
    except:
      pass
    pos_line.append([float(pos_line_str[0]), float(pos_line_str[1])])
    vel_line.append([float(vel_line_str[0]), float(vel_line_str[1])])
  pos_from_file.append(pos_line)
  vel_from_file.append(vel_line)
  pos_line = []
  vel_line = []

In [0]:
pos = []
vel =[]
for i in range(len(pos_from_file)):
    pos.append(np.asarray(pos_from_file[i]))
    vel.append(np.asarray(vel_from_file[i]))
pos = np.asarray(pos)
vel = np.asarray(vel)
x = pos[:,:,0]
y = pos[:,:,1]
vx = vel[:,:,0]
vy = vel[:,:,1]

In [0]:
# CREATING FEATURES FOR ALL 32 AGENTS:
# EACH AGENT LEARNS A MODEL. IT HAS 66 FEATURES: X & Y coordinates of all 32 agents + VX & VY of itself

vxx = {}
vyy = {}
for i in range(32):
  vxx[i] = vx[:,i].reshape(-1,1)
  vyy[i] = vy[:,i].reshape(-1,1)

agent = {}
for i in range(32):
  agent[i] = np.append(x,y,axis = 1)
  agent[i] = np.append(agent[i],vxx[i],axis = 1)
  agent[i] = np.append(agent[i],vyy[i],axis = 1)

# agent[i] contains 3000 data points each having 66 features for training

In [0]:
# CREATE TRAINING AND TESTING DATASET: 2000 DATAPOINTS FOR TRAINING & 1000 DATAPOINTS FOR TESTING
agent_train_data = {}
agent_test_data = {}
for i in range(32):
  agent_train_data[i] = agent[i][:2000,:]
  agent_test_data[i] = agent[i][2000:,:]

### **TRAINING SINDy MODEL**

In [0]:
# POLYNOMIAL LIBRARY IS USED TO DEFINE THE SINDY MODEL
feature_library = poly_library

In [0]:
dt = 0.0001
list_models = []
for i in range(32):
    model= ps.SINDy(feature_library = feature_library)
    model.fit(agent_train_data[i],t = dt)
    list_models.append(model)

### **ACCESSING DERIVATIVES ON TEST DATA**

In [0]:
x_dot_test_computed = [None]*32
for i in range(32):
  x_dot_test_computed[i] = list_models[i].differentiate(agent_test_data[i], t=dt)
x_dot_test_computed = np.asarray(x_dot_test_computed)

In [0]:
x_dot_test_predicted = [None]*32
for i in range(32):
  x_dot_test_predicted[i] = list_models[i].differentiate(agent_test_data[i], t=dt)
  #print('Model score: %f' % list_models[i].score(agent_test_data[i], t=dt))
x_dot_test_predicted = np.asarray(x_dot_test_predicted)

### **COMPARISION OF PREDICTIONS AND GROUND TRUTH**

In [0]:
import matplotlib.pyplot as plt
NUMBER_OF_AGENTS = 32
delta_x = [None]*NUMBER_OF_AGENTS
delta_y = [None]*NUMBER_OF_AGENTS
x0 = [None]*NUMBER_OF_AGENTS
y0 = [None]*NUMBER_OF_AGENTS
xt = [None]*NUMBER_OF_AGENTS
yt = [None]*NUMBER_OF_AGENTS
truth_x = [None]*NUMBER_OF_AGENTS
truth_y = [None]*NUMBER_OF_AGENTS
pred_x = [None]*NUMBER_OF_AGENTS
pred_y = [None]*NUMBER_OF_AGENTS
for i in range(NUMBER_OF_AGENTS):
  delta_x[i] = x_dot_test_predicted[i][:,0+i]*dt
  delta_y[i] = x_dot_test_predicted[i][:,NUMBER_OF_AGENTS+i]*dt
  x0[i] = agent_test_data[i][0+i]
  y0[i] = agent_test_data[i][NUMBER_OF_AGENTS+i]
  xt[i] = agent_test_data[i][1:,0+i]
  yt[i] = agent_test_data[i][1:,NUMBER_OF_AGENTS+i]

  del_x = delta_x[0+i]
  del_y = delta_y[0+i]
  x00 = x0[0+i][0+i]
  y00 = y0[0+i][NUMBER_OF_AGENTS+i]

  xs = []
  for dx in del_x:
    x = x00
    newx = x+dx
    xs.append(newx)
    x00 = newx

  ys = []
  for dy in del_y:
    y = y00
    newy = y+dy
    ys.append(newy)
    y00 = newy

  truth_x[i]=xt[i]
  pred_x[i]=xs  
  truth_y[i]=yt[i]
  pred_y[i]=ys
  plt.plot(xs,label = "Predicted")
  plt.plot(xt[i],label = "Ground Truth")
  plt.legend()
  plt.show()
pred_x = np.array(pred_x).T
pred_y = np.array(pred_y).T
truth_x = np.array(truth_x).T
truth_y = np.array(truth_y).T

### **MEAN FIELD ANALYSIS**

In [0]:
AGENT_COUNT =32
x = pred_x
y = pred_y
x_gt = truth_x
y_gt = truth_y
pred_x = pred_x[0:len(pred_x)-1,:]
pred_y = pred_y[0:len(pred_y)-1,:]
mean_x = np.sum(x,axis=1)/AGENT_COUNT
mean_y = np.sum(y,axis=1)/AGENT_COUNT
mean_x_gt = np.sum(x_gt,axis=1)/AGENT_COUNT
mean_y_gt = np.sum(y_gt,axis=1)/AGENT_COUNT

In [0]:
mean_field_distance = np.sqrt(((mean_x-mean_x_gt)**2 +(mean_y-mean_y_gt)**2 ))
title = "Mean Field Error [Trained on Transients]"
xlabel = "Samples in Test Data"
ylabel = "Distance between Predicted and True Mean Field"
ts = [t for t in range(len(mean_field_distance))]
df = pd.DataFrame(list(zip(ts,mean_field_distance)),columns =[xlabel, ylabel]) 
fig = plotx.line(df, x=xlabel, y=ylabel)
fig.update_layout(
    title={
        'text': title,
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.update_layout(
    autosize=False,
    width=500,
    height=500,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
    paper_bgcolor="White",
)
fig.update_yaxes(range=[0, 5])
fig.show()

### **GENERATING A VIDEO TO OBSERVE THE PREDICTED SWARM BEHAVIOUR**

In [0]:
def make_animation(AGENT_COUNT, trueX,trueY,predX,predY,total_frame, save_name, include_true=True, include_pred=False, Compare=False, save=0):
  ############ REPLAYING ANIMATION ############


  # Making legends
  blue_dot = mlines.Line2D([], [], color='blue', marker='.', linestyle='None',
                            markersize=10, label='Prediction')
  red_dot = mlines.Line2D([], [], color='red', marker='.', linestyle='None',
                            markersize=10, label='Ground Truth')
  blue_cross = mlines.Line2D([], [], color='blue', marker='+', linestyle='None',
                          markersize=10, label='Pred Mean Field')
  red_cross = mlines.Line2D([], [], color='red', marker='+', linestyle='None',
                          markersize=10, label='True Mean Field')
  legend_handle = []
  if include_true:
    legend_handle.append(red_dot)
    legend_handle.append(red_cross)
  if include_pred:
    legend_handle.append(blue_dot)
    legend_handle.append(blue_cross)
  
  plt.legend(handles=legend_handle)

  pred_len = len(trueX)
  print("Predicted number of time steps: ", pred_len)

  x = predX
  y = predY
  x_gt = trueX
  y_gt = trueY

  mean_x = np.sum(x,axis=1)/AGENT_COUNT
  mean_y = np.sum(y,axis=1)/AGENT_COUNT
  mean_x_gt = np.sum(x_gt,axis=1)/AGENT_COUNT
  mean_y_gt = np.sum(y_gt,axis=1)/AGENT_COUNT




  x_min = np.amin(x)-0.5
  x_max = np.amax(x)+0.5
  y_min = np.amin(y)-0.5
  y_max = np.amax(y)+0.5

  fig = plt.figure()
  fig.set_size_inches(8, 8)
  ax = plt.axes(xlim=(x_min,x_max),ylim=(y_min,y_max))


  # initialize animation parameters
  if Compare:
    gg = [Line2D([x_gt[0][0], x[0][0]],[y_gt[0][0], y[0][0]]) for i in range(AGENT_COUNT)]
    for j in range(AGENT_COUNT):
      gg[j], = ax.plot([x_gt[0][j], x[0][j]],[y_gt[0][j], y[0][j]], c='mediumspringgreen',marker='.')

  if include_pred:
    p, = ax.plot(x[0],y[0], c='blue',marker='.',linestyle='None')
    p_mean, = ax.plot(mean_x[0],mean_y[0], c='blue',marker='+',linestyle='None')

  if include_true:
    t, = ax.plot(x_gt[0],y_gt[0], c='red',marker='.',linestyle='None')
    t_mean, = ax.plot(mean_x_gt[0],mean_y_gt[0], c='red',marker='+',linestyle='None')


  def animate(i):
      plt.legend(handles=legend_handle)
      if Compare:
        for j in range(AGENT_COUNT):
          gg[j].set_data([x_gt[i][j], x[i][j]],[y_gt[i][j], y[i][j]])

      if include_pred:
        p.set_data(x[i],y[i])
        p_mean.set_data(mean_x[i],mean_y[i])
      if include_true:
        t.set_data(x_gt[i],y_gt[i])
        t_mean.set_data(mean_x_gt[i],mean_y_gt[i])

      if include_pred and include_true:
        return p, t, p_mean, t_mean,
      elif include_pred:
        return p, p_mean,
      else:
        return t, t_mean,

  # call the animator.  blit=True means only re-draw the parts that have changed.
  anim = animation.FuncAnimation(fig, animate, frames=total_frame, interval=20, blit=True)
  if save:
    anim.save(save_name, writer=animation.FFMpegWriter(fps=20))
  return anim

In [0]:
import matplotlib.lines as mlines
SAVE_NAME = 'sindy_1.mp4'
TOTAL_FRAME = 999
anim = make_animation(AGENT_COUNT = 32, trueX = truth_x,trueY = truth_y,predX = pred_x,predY = pred_y,
                      total_frame = TOTAL_FRAME, save_name = SAVE_NAME, 
                      include_true=True, include_pred=True, Compare=False, save=1)
rc('animation', html='jshtml')
anim

## **SINDy MODEL FOR TRANSIENTS**

### **SETTING UP DATA**

In [0]:
# READING THE DATA FROM A CSV FILE INTO LISTS - LOAD THE FILE PATH OF TRANSIENT DATA
file_path = "/content/drive/My Drive/522_PROJECT/diff_init_04112020/10_agent_3.csv"
my_data = []
with open(file_path) as csvDataFile:
  csvReader = csv.reader(csvDataFile)
  for row in csvReader:
    my_data.append(row)
pos_from_file = []
vel_from_file =[]
pos_line = []
vel_line = []

for i in range(len(my_data)):
  for j in range(int(len(my_data[0])/2)):
    try:
      pos_line_str = my_data[i][j*2].lstrip('[').rstrip(']').split()
      vel_line_str = my_data[i][j*2+1].lstrip('[').rstrip(']').split()
    except:
      pass
    pos_line.append([float(pos_line_str[0]), float(pos_line_str[1])])
    vel_line.append([float(vel_line_str[0]), float(vel_line_str[1])])
  pos_from_file.append(pos_line)
  vel_from_file.append(vel_line)
  pos_line = []
  vel_line = []

In [0]:
# POSITION AND VELOCITIES OF THE AGENTS
pos = []
vel =[]
for i in range(len(pos_from_file)):
    pos.append(np.asarray(pos_from_file[i]))
    vel.append(np.asarray(vel_from_file[i]))
pos = np.asarray(pos)
vel = np.asarray(vel)

In [0]:
# UNDERSTANDING THE DATA AND GETTING X,Y COORDINATES AND VX,VY COMPONENTS OF VELOCITY
x = pos[:,:,0]
y = pos[:,:,1]
vx = vel[:,:,0]
vy = vel[:,:,1]
TOTAL_SAMPLES = len(x)
NUMBER_OF_AGENTS = 10
print("Number of samples in the time series data:",TOTAL_SAMPLES)
print("Number of agents:",NUMBER_OF_AGENTS)

Number of samples in the time series data: 250
Number of agents: 10


In [0]:
# CREATING FEATURES FOR ALL AGENTS:
# EACH AGENT LEARNS A MODEL. IT HAS 22 FEATURES: X & Y coordinates of all 10 agents + VX & VY of itself

vxx = {}
vyy = {}
for i in range(NUMBER_OF_AGENTS):
  vxx[i] = vx[:,i].reshape(-1,1)
  vyy[i] = vy[:,i].reshape(-1,1)

agent = {}
for i in range(NUMBER_OF_AGENTS):
  agent[i] = np.append(x,y,axis = 1)
  agent[i] = np.append(agent[i],vxx[i],axis = 1)
  agent[i] = np.append(agent[i],vyy[i],axis = 1)
# agent[i] contains 250 data points each having 22 features for training
print("Training data is a numpy array of shape:",agent[0].shape)

Training data is a numpy array of shape: (250, 22)


In [0]:
# CREATE TRAINING AND TESTING DATASET: 200 DATAPOINTS FOR TRAINING & 50 DATAPOINTS FOR TESTING
TRAIN_TEST_SPLIT = 100
agent_train_data = {}
agent_test_data = {}
for i in range(NUMBER_OF_AGENTS):
  agent_train_data[i] = agent[i][:TRAIN_TEST_SPLIT,:]
  agent_test_data[i] = agent[i][TRAIN_TEST_SPLIT:,:]

### **TRAINING SINDy MODEL**

In [0]:
# Custom Library for building Sparse Regression Model
def choose_function_library(library_index):
    #CHOICE 1: POLYNOMIAL LIBRARY 
    poly_library = ps.PolynomialLibrary
    #-------------------------------------------------------------------
    #CHOICE 2: POLYNOMIAL LIBRARY WITHOUT INTERACTION TERMS
    poly_library_1 = ps.PolynomialLibrary(include_interaction=False)
    #-------------------------------------------------------------------
    #CHOICE 3: FOURIER LIBRARY
    fourier_library = ps.FourierLibrary(n_frequencies=3)
    #-------------------------------------------------------------------
    #CHOICE 4: FULLY CUSTOM LIBRARY 1
    library_functions_1 = [
        lambda x : x,
        lambda x : np.sin(x),
        lambda x : np.cos(x),
        lambda x : np.exp(x),
        lambda x : 1./x,
        lambda x : np.sqrt(abs(x)),
        lambda x : np.log(abs(x)),
        lambda x : x*x*x,
        lambda x : np.abs(x),
        lambda x : np.tanh(x),
        lambda x : x*x,
        lambda x : (1-x*x)*x]
    custom_library_1 = ps.CustomLibrary(library_functions=library_functions_1)
    #-------------------------------------------------------------------
    #CHOICE 5: FULLY CUSTOM LIBRARY 2
    library_functions_2 = [
        lambda x,y : x*y,
        lambda x,y : x-y,
        lambda x,y : x/y,
        lambda x : 1./x,
        lambda x,y : x*y*y ,
        lambda x,y : x*x*y*y*y,
        lambda x : x*x*x,
        lambda x : np.exp(x),
        lambda x : np.tanh(x)]
    custom_library_2 = ps.CustomLibrary(library_functions=library_functions_2)

    if library_index == 1:
       feature_library = poly_library
       mode_ = "POLYNOMIAL LIBRARY"
    elif library_index == 2:
         feature_library = poly_library_1
         mode_ = "POLYNOMIAL LIBRARY WITHOUT INTERACTION TERMS"
    elif library_index == 3:
         feature_library = fourier_library
         mode_ = "FOURIER LIBRARY"
    elif library_index == 4:
         feature_library = custom_library_1
         mode_ = "FULLY CUSTOM LIBRARY 1 (WITH MEMEBER FUNCTIONS)"
    elif library_index == 5:
         feature_library = custom_library_2 
         mode_ = "FULLY CUSTOM LIBRARY 2 (WITHOUT MEMBER FUNCTIONS)"
    return feature_library,mode_

In [0]:
# CHOOSE FUNCTION LIBRARY OF SINDY FOR BUILDING MODELS
#------------------------------------------------------
#CHOICE 1: POLYNOMIAL LIBRARY
#CHOICE 2: POLYNOMIAL LIBRARY WITHOUT INTERACTION TERMS
#CHOICE 3: FOURIER LIBRARY
#CHOICE 4: FULLY CUSTOM LIBRARY 1 (WITHOUT INTERACTION FUNCTIONS)
#CHOICE 5: FULLY CUSTOM LIBRARY 2 (WITH INTERACTION FUNCTIONS)
choice = 4
feature_library,mode_ = choose_function_library(choice)
print("Chosen Library is:",mode_)

Chosen Library is: FULLY CUSTOM LIBRARY 1 (WITH MEMEBER FUNCTIONS)


In [0]:
# SPARSE REGRESSION MODEL BUILDING USING THE FUNCTION LIBRARY AND AGENT TRAIN DATA
dt = 0.0001
list_models = []
for i in range(NUMBER_OF_AGENTS):
    if choice == 1:
      model = ps.SINDy()
    else:
      model= ps.SINDy(feature_library = feature_library)
    model.fit(agent_train_data[i],t = dt)
    list_models.append(model)

### **ACCESSING DERIVATIVES ON TEST DATA**

In [0]:
x_dot_test_computed = [None]*32
for i in range(32):
  x_dot_test_computed[i] = list_models[i].differentiate(agent_test_data[i], t=dt)
x_dot_test_computed = np.asarray(x_dot_test_computed)

In [0]:
# ACCESSING THE DERIVATIVES OF THE TEST DATA : PREDICTED DERIVATIVES
x_dot_test_predicted = [None]*NUMBER_OF_AGENTS
for i in range(NUMBER_OF_AGENTS):
  x_dot_test_predicted[i] = list_models[i].differentiate(agent_test_data[i],t=dt)
x_dot_test_predicted = np.asarray(x_dot_test_predicted)

### **COMPARISION OF PREDICITONS AND GROUND TRUTH**

In [0]:
import matplotlib.pyplot as plt
delta_x = [None]*NUMBER_OF_AGENTS
delta_y = [None]*NUMBER_OF_AGENTS
x0 = [None]*NUMBER_OF_AGENTS
y0 = [None]*NUMBER_OF_AGENTS
xt = [None]*NUMBER_OF_AGENTS
yt = [None]*NUMBER_OF_AGENTS
truth_x = [None]*NUMBER_OF_AGENTS
truth_y = [None]*NUMBER_OF_AGENTS
pred_x = [None]*NUMBER_OF_AGENTS
pred_y = [None]*NUMBER_OF_AGENTS
for i in range(NUMBER_OF_AGENTS):
  delta_x[i] = x_dot_test_predicted[i][:,0+i]*dt
  delta_y[i] = x_dot_test_predicted[i][:,NUMBER_OF_AGENTS+i]*dt
  x0[i] = agent_test_data[i][0+i]
  y0[i] = agent_test_data[i][NUMBER_OF_AGENTS+i]
  xt[i] = agent_test_data[i][1:,0+i]
  yt[i] = agent_test_data[i][1:,NUMBER_OF_AGENTS+i]

  del_x = delta_x[0+i]
  del_y = delta_y[0+i]
  x00 = x0[0+i][0+i]
  y00 = y0[0+i][NUMBER_OF_AGENTS+i]

  xs = []
  for dx in del_x:
    x = x00
    newx = x+dx
    xs.append(newx)
    x00 = newx

  ys = []
  for dy in del_y:
    y = y00
    newy = y+dy
    ys.append(newy)
    y00 = newy

  truth_x[i]=xt[i]
  pred_x[i]=xs  
  truth_y[i]=yt[i]
  pred_y[i]=ys
  # plt.plot(xs,label = "Predicted")
  # plt.plot(xt[i],label = "Ground Truth")
  # plt.legend()
  # plt.show()
pred_x = np.array(pred_x).T
pred_y = np.array(pred_y).T
truth_x = np.array(truth_x).T
truth_y = np.array(truth_y).T

### **MEAN FIELD ANALYSIS**

In [0]:
AGENT_COUNT =32
x = pred_x
y = pred_y
x_gt = truth_x
y_gt = truth_y
pred_x = pred_x[0:len(pred_x)-1,:]
pred_y = pred_y[0:len(pred_y)-1,:]
mean_x = np.sum(x,axis=1)/AGENT_COUNT
mean_y = np.sum(y,axis=1)/AGENT_COUNT
mean_x_gt = np.sum(x_gt,axis=1)/AGENT_COUNT
mean_y_gt = np.sum(y_gt,axis=1)/AGENT_COUNT

In [0]:
mean_field_distance = np.sqrt(((mean_x-mean_x_gt)**2 +(mean_y-mean_y_gt)**2 ))
title = "Mean Field Error [Trained on Transients]"
xlabel = "Samples in Test Data"
ylabel = "Distance between Predicted and True Mean Field"
ts = [t for t in range(len(mean_field_distance))]
df = pd.DataFrame(list(zip(ts,mean_field_distance)),columns =[xlabel, ylabel]) 
fig = plotx.line(df, x=xlabel, y=ylabel)
fig.update_layout(
    title={
        'text': title,
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.update_layout(
    autosize=False,
    width=500,
    height=500,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
    paper_bgcolor="White",
)
fig.update_yaxes(range=[0, 5])
fig.show()

### **GENERATING A VIDEO TO OBSERVE THE PREDICTED SWARM BEHAVIOUR**

In [0]:
def make_animation(AGENT_COUNT, trueX,trueY,predX,predY,total_frame, save_name, include_true=True, include_pred=False, Compare=False, save=0):
  ############ REPLAYING ANIMATION ############


  # Making legends
  blue_dot = mlines.Line2D([], [], color='blue', marker='.', linestyle='None',
                            markersize=10, label='Prediction')
  red_dot = mlines.Line2D([], [], color='red', marker='.', linestyle='None',
                            markersize=10, label='Ground Truth')
  blue_cross = mlines.Line2D([], [], color='blue', marker='+', linestyle='None',
                          markersize=10, label='Pred Mean Field')
  red_cross = mlines.Line2D([], [], color='red', marker='+', linestyle='None',
                          markersize=10, label='True Mean Field')
  legend_handle = []
  if include_true:
    legend_handle.append(red_dot)
    legend_handle.append(red_cross)
  if include_pred:
    legend_handle.append(blue_dot)
    legend_handle.append(blue_cross)
  
  plt.legend(handles=legend_handle)

  pred_len = len(trueX)
  print("Predicted number of time steps: ", pred_len)

  x = predX
  y = predY
  x_gt = trueX
  y_gt = trueY

  mean_x = np.sum(x,axis=1)/AGENT_COUNT
  mean_y = np.sum(y,axis=1)/AGENT_COUNT
  mean_x_gt = np.sum(x_gt,axis=1)/AGENT_COUNT
  mean_y_gt = np.sum(y_gt,axis=1)/AGENT_COUNT




  x_min = np.amin(x)-0.5
  x_max = np.amax(x)+0.5
  y_min = np.amin(y)-0.5
  y_max = np.amax(y)+0.5

  fig = plt.figure()
  fig.set_size_inches(8, 8)
  ax = plt.axes(xlim=(x_min,x_max),ylim=(y_min,y_max))


  # initialize animation parameters
  if Compare:
    gg = [Line2D([x_gt[0][0], x[0][0]],[y_gt[0][0], y[0][0]]) for i in range(AGENT_COUNT)]
    for j in range(AGENT_COUNT):
      gg[j], = ax.plot([x_gt[0][j], x[0][j]],[y_gt[0][j], y[0][j]], c='mediumspringgreen',marker='.')

  if include_pred:
    p, = ax.plot(x[0],y[0], c='blue',marker='.',linestyle='None')
    p_mean, = ax.plot(mean_x[0],mean_y[0], c='blue',marker='+',linestyle='None')

  if include_true:
    t, = ax.plot(x_gt[0],y_gt[0], c='red',marker='.',linestyle='None')
    t_mean, = ax.plot(mean_x_gt[0],mean_y_gt[0], c='red',marker='+',linestyle='None')


  def animate(i):
      plt.legend(handles=legend_handle)
      if Compare:
        for j in range(AGENT_COUNT):
          gg[j].set_data([x_gt[i][j], x[i][j]],[y_gt[i][j], y[i][j]])

      if include_pred:
        p.set_data(x[i],y[i])
        p_mean.set_data(mean_x[i],mean_y[i])
      if include_true:
        t.set_data(x_gt[i],y_gt[i])
        t_mean.set_data(mean_x_gt[i],mean_y_gt[i])

      if include_pred and include_true:
        return p, t, p_mean, t_mean,
      elif include_pred:
        return p, p_mean,
      else:
        return t, t_mean,

  # call the animator.  blit=True means only re-draw the parts that have changed.
  anim = animation.FuncAnimation(fig, animate, frames=total_frame, interval=20, blit=True)
  if save:
    anim.save(save_name, writer=animation.FFMpegWriter(fps=20))
  return anim

### **EXTENDING MODEL PREDICITONS TO STEADY STATE**

In [0]:
# ACCESSING THE DERIVATIVES OF THE TEST DATA : PREDICTED DERIVATIVES 1000 TIME STEPS AHEAD INTO STEADY STATE
x_dot_test_predicted = [None]*NUMBER_OF_AGENTS
for i in range(NUMBER_OF_AGENTS):
  x_dot_test_predicted[i] = list_models[i].differentiate(agent_test_data[i],t=dt)
x_dot_test_predicted = np.asarray(x_dot_test_predicted)

In [0]:
import matplotlib.pyplot as plt
delta_x = [None]*NUMBER_OF_AGENTS
delta_y = [None]*NUMBER_OF_AGENTS
x0 = [None]*NUMBER_OF_AGENTS
y0 = [None]*NUMBER_OF_AGENTS
xt = [None]*NUMBER_OF_AGENTS
yt = [None]*NUMBER_OF_AGENTS
truth_x = [None]*NUMBER_OF_AGENTS
truth_y = [None]*NUMBER_OF_AGENTS
pred_x = [None]*NUMBER_OF_AGENTS
pred_y = [None]*NUMBER_OF_AGENTS
for i in range(NUMBER_OF_AGENTS):
  delta_x[i] = x_dot_test_predicted[i][:,0+i]*dt
  delta_y[i] = x_dot_test_predicted[i][:,NUMBER_OF_AGENTS+i]*dt
  x0[i] = agent_test_data[i][0+i]
  y0[i] = agent_test_data[i][NUMBER_OF_AGENTS+i]
  xt[i] = agent_test_data[i][1:,0+i]
  yt[i] = agent_test_data[i][1:,NUMBER_OF_AGENTS+i]

  del_x = delta_x[0+i]
  del_y = delta_y[0+i]
  x00 = x0[0+i][0+i]
  y00 = y0[0+i][NUMBER_OF_AGENTS+i]

  xs = []
  for dx in del_x:
    x = x00
    newx = x+dx
    xs.append(newx)
    x00 = newx

  ys = []
  for dy in del_y:
    y = y00
    newy = y+dy
    ys.append(newy)
    y00 = newy

  truth_x[i]=xt[i]
  pred_x[i]=xs  
  truth_y[i]=yt[i]
  pred_y[i]=ys
  # plt.plot(xs,label = "Predicted")
  # plt.plot(xt[i],label = "Ground Truth")
  # plt.legend()
  # plt.show()
pred_x = np.array(pred_x).T
pred_y = np.array(pred_y).T
truth_x = np.array(truth_x).T
truth_y = np.array(truth_y).T

In [0]:
SAVE_NAME = 'sindy_2.mp4'
TOTAL_FRAME = 999
anim = make_animation(AGENT_COUNT = 10, trueX = truth_x,trueY = truth_y,predX = pred_x,predY = pred_y,
                      total_frame = TOTAL_FRAME, save_name = SAVE_NAME, 
                      include_true=False, include_pred=True, Compare=False, save=1)
rc('animation', html='jshtml')
anim